In [3]:
import cv2
import os
import random

def generate_random_block_positions(image_size, block_size, num_blocks):
    """
    Generates random positions for smaller mosaic blocks.

    Parameters:
    - image_size (tuple): (width, height) of the image.
    - block_size (tuple): (width, height) of each block.
    - num_blocks (int): Number of blocks to generate.

    Returns:
    - list of tuples: List of (x, y) coordinates for the top-left corner of each block.
    """
    img_width, img_height = image_size
    block_width, block_height = block_size
    positions = set()  # Use a set to avoid overlapping blocks

    max_x = img_width - block_width
    max_y = img_height - block_height

    if max_x < 0 or max_y < 0:
        raise ValueError("Block size is larger than the image size. Cannot generate positions.")

    while len(positions) < num_blocks:
        x = random.randint(0, max_x)
        y = random.randint(0, max_y)
        positions.add((x, y))  # Ensure uniqueness of positions

    return list(positions)

def apply_random_mosaic_blocks(image, positions, block_size, color):
    """
    Applies multiple randomly positioned mosaic blocks to an image.

    Parameters:
    - image (np.array): Input image.
    - positions (list of tuples): List of (x, y) coordinates for each block.
    - block_size (tuple): (width, height) of each block.
    - color (tuple): RGB color of the mosaic in (B, G, R) format.

    Returns:
    - Processed image with mosaic blocks.
    """
    for (x, y) in positions:
        block_width, block_height = block_size
        image[y:y + block_height, x:x + block_width] = color
    return image

def process_images_in_directory(input_base_directory, output_base_directory, target_size=(192, 192), total_mosaic_size=(80, 80), block_size=(20, 20)):
    """
    Processes all images in a directory by resizing and applying randomly positioned smaller mosaic blocks.

    Parameters:
    - input_base_directory (str): Directory containing input images.
    - output_base_directory (str): Directory to save processed images.
    - target_size (tuple): (width, height) to resize the images.
    - total_mosaic_size (tuple): Total area (width, height) covered by mosaic blocks.
    - block_size (tuple): (width, height) of each individual block.
    """
    colors = {
        'blue': (255, 0, 0),
        'yellow': (0, 255, 255),
        'red': (0, 0, 255),
        'black': (0, 0, 0),
        'white': (255, 255, 255)
    }

    # Ensure the output directory exists
    os.makedirs(output_base_directory, exist_ok=True)

    # Iterate through all files in the input directory
    for root, _, files in os.walk(input_base_directory):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_image_path = os.path.join(root, filename)

                # Load the image
                image = cv2.imread(input_image_path)
                if image is None:
                    print(f"Error loading image: {input_image_path}. Skipping...")
                    continue

                # Resize the image
                resized_image = resize_image(image, target_size)

                # Calculate number of blocks to cover the total mosaic size
                num_blocks = (total_mosaic_size[0] // block_size[0]) * (total_mosaic_size[1] // block_size[1])

                # Generate random positions for the blocks
                positions = generate_random_block_positions(target_size, block_size, num_blocks)

                # Apply the mosaic effect with each color
                for color_name, color in colors.items():
                    # Apply random mosaic blocks
                    result = apply_random_mosaic_blocks(resized_image.copy(), positions, block_size, color)

                    # Create a subdirectory for the color-specific output
                    relative_dir = os.path.relpath(root, input_base_directory)  # Preserve relative directory structure
                    color_output_path = os.path.join(output_base_directory, relative_dir, f"{color_name}")
                    os.makedirs(color_output_path, exist_ok=True)

                    # Save the processed image
                    output_file = os.path.join(color_output_path, filename)
                    cv2.imwrite(output_file, result)
                    print(f"Processed and saved: {output_file} with color {color_name}")

def resize_image(image, target_size):
    """
    Resizes the image to a fixed target size.

    Parameters:
    - image (np.array): Input image.
    - target_size (tuple): (width, height) to resize the image.

    Returns:
    - Resized image.
    """
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

# Example usage
if __name__ == "__main__":
    # Define the input and output directories
    input_base_directory = './ori_data_try'  # Directory containing the original images
    output_base_directory = './processed_images'  # Directory to save processed images

    # Process all images in the input directory
    process_images_in_directory(
        input_base_directory=input_base_directory,
        output_base_directory=output_base_directory,
        target_size=(192, 192),  # Resize images to 192x192
        total_mosaic_size=(80, 80),  # Total area covered by mosaic blocks
        block_size=(20, 20)  # Size of each individual block
    )


Processed and saved: ./processed_images/./blue/original_p1.png with color blue
Processed and saved: ./processed_images/./yellow/original_p1.png with color yellow
Processed and saved: ./processed_images/./red/original_p1.png with color red
Processed and saved: ./processed_images/./black/original_p1.png with color black
Processed and saved: ./processed_images/./white/original_p1.png with color white
Processed and saved: ./processed_images/./blue/oblackp2.png with color blue
Processed and saved: ./processed_images/./yellow/oblackp2.png with color yellow
Processed and saved: ./processed_images/./red/oblackp2.png with color red
Processed and saved: ./processed_images/./black/oblackp2.png with color black
Processed and saved: ./processed_images/./white/oblackp2.png with color white
